PARTE 1

In [ ]:
import os
# Forzamos a TensorFlow a usar el motor antiguo de Keras 2, ya que no esta disponible para el kernel de python 3.14.2
os.environ["TF_USE_LEGACY_KERAS"] = "1"

import tensorflow as tf
import tf_keras as keras  # Usamos el paquete de compatibilidad
import transformers

print(f"Versión de TF: {tf.__version__}")
# Debería mostrar algo como 2.16.x o 2.17.x pero usando el motor de tf-keras
# Despues de correr esto podemos ejecutar el siguiente fragmento de codigo

2026-01-25 02:41:50.421368: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Versión de TF: 2.16.2


In [2]:

import pandas as pd
import numpy as np
import re
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from sentence_transformers import SentenceTransformer

# ==========================================
# 0. Carga y Preprocesamiento de Datos
# ==========================================
df = pd.read_csv('retail_reviews.csv')

def clean_text(text):
    if pd.isna(text): return ""
    # Eliminar caracteres especiales tipo #@#&* y números
    text = re.sub(r'[^a-zA-ZáéíóúñÁÉÍÓÚÑ ]', '', text)
    # Convertir a minúsculas y quitar espacios extra
    return text.lower().strip()

df['text_clean'] = df['text'].apply(clean_text)
# Eliminar filas vacías tras la limpieza
df = df[df['text_clean'] != ""]

# ==========================================
# 1.1 Análisis de Embeddings (Word2Vec)
# ==========================================

# Tokenización para Word2Vec
tokenized_corpus = [doc.split() for doc in df['text_clean']]

# Entrenamiento del modelo Word2Vec
# vector_size: dimensión del vector, window: contexto, min_count: frecuencia mínima
w2v_model = Word2Vec(sentences=tokenized_corpus, vector_size=100, window=5, min_count=1, workers=4)

print("--- 1.1.a) Términos Similares ---")
palabras_clave = ["defectuoso", "rápido"]
for palabra in palabras_clave:
    if palabra in w2v_model.wv:
        similares = w2v_model.wv.most_similar(palabra, topn=5)
        print(f"\nSimilares a '{palabra}':")
        for p, sim in similares:
            print(f" - {p}: {sim:.4f}")

print("\n--- 1.1.c) Álgebra Vectorial (Analogía Retail) ---")
# Analogía: "excelente" - "positivo" + "negativo" debería tender a algo como "malo" o "pésimo"
try:
    resultado_algebra = w2v_model.wv.most_similar(positive=['excelente', 'negativo'], negative=['positivo'], topn=1)
    print(f"Operación: 'excelente' - 'positivo' + 'negativo'")
    print(f"Resultado semántico: {resultado_algebra[0][0]} (Similitud: {resultado_algebra[0][1]:.4f})")
except KeyError as e:
    print(f"Error en analogía: {e}")

# ==========================================
# 1.2 Clasificación de Texto
# ==========================================

# Preparación de etiquetas (Label Encoding manual para Positivo/Negativo)
df['label'] = df['sentiment'].map({'Positivo': 1, 'Negativo': 0})
df = df.dropna(subset=['label']) # Limpiar si hay etiquetas mal formadas

X_train, X_test, y_train, y_test = train_test_split(
    df['text_clean'], df['label'], test_size=0.2, random_state=42
)

# --- Enfoque A: Baseline TF-IDF + Regresión Logística ---
tfidf = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

model_lr = LogisticRegression()
model_lr.fit(X_train_tfidf, y_train)
y_pred_tfidf = model_lr.predict(X_test_tfidf)
f1_tfidf = f1_score(y_test, y_pred_tfidf)

# --- Enfoque B: BERT Embeddings + Regresión Logística ---
# Usamos un modelo ligero de BERT (paraphrase-multilingual-MiniLM-L12-v2)
print("\nGenerando embeddings de BERT (esto puede tardar un poco)...")
bert_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

X_train_bert = bert_model.encode(X_train.tolist())
X_test_bert = bert_model.encode(X_test.tolist())

model_bert = LogisticRegression()
model_bert.fit(X_train_bert, y_train)
y_pred_bert = model_bert.predict(X_test_bert)
f1_bert = f1_score(y_test, y_pred_bert)

# ==========================================
# Reporte Final
# ==========================================
print("\n" + "="*30)
print("REPORTE DE COMPARACIÓN")
print("="*30)
print(f"F1-Score TF-IDF (Baseline): {f1_tfidf:.4f}")
print(f"F1-Score BERT Embeddings:    {f1_bert:.4f}")
print("-" * 30)

# Identificar un caso donde TF-IDF falla y BERT acierta
print("\n--- Análisis de Fallos Específicos ---")
for i in range(len(y_test)):
    idx = y_test.index[i]
    if y_pred_tfidf[i] != y_test.iloc[i] and y_pred_bert[i] == y_test.iloc[i]:
        print(f"Texto: '{df.loc[idx, 'text']}'")
        print(f"Real: {y_test.iloc[i]} | TF-IDF predijo: {y_pred_tfidf[i]} | BERT predijo: {y_pred_bert[i]}")
        break

--- 1.1.a) Términos Similares ---

Similares a 'defectuoso':
 - insatisfecho: 0.9882
 - una: 0.9869
 - total: 0.9857
 - sucio: 0.9847
 - empaque: 0.9816

Similares a 'rápido':
 - recomendado: 0.9850
 - llegó: 0.9830
 - estado: 0.9817
 - eficaz: 0.9810
 - y: 0.9784

--- 1.1.c) Álgebra Vectorial (Analogía Retail) ---
Error en analogía: "Key 'negativo' not present in vocabulary"

Generando embeddings de BERT (esto puede tardar un poco)...

REPORTE DE COMPARACIÓN
F1-Score TF-IDF (Baseline): 0.9000
F1-Score BERT Embeddings:    0.8924
------------------------------

--- Análisis de Fallos Específicos ---
Texto: '  Producto duradero pero el precio es elevado.  '
Real: 0.0 | TF-IDF predijo: 1.0 | BERT predijo: 0.0


PARTE 2

In [4]:
# 1. Instalación de librerías (Ejecutar en la primera celda)
# !pip install bertopic sentence-transformers pandas

import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
import re

# ==========================================
# 1. Carga y Limpieza (Igual que Parte 1)
# ==========================================
df = pd.read_csv('retail_reviews.csv')

# Limpieza básica para quitar ruido visual
def clean_text(text):
    if pd.isna(text): return ""
    text = re.sub(r'[^a-zA-ZáéíóúñÁÉÍÓÚÑ ]', '', text)
    return text.lower().strip()

df['text_clean'] = df['text'].apply(clean_text)
docs = df[df['text_clean'] != ""]['text_clean'].tolist()

# ==========================================
# 2. Configuración del Pipeline BERTopic
# ==========================================

# Paso 1: Embeddings (Multilingüe para Español)
# Usamos un modelo que soporte español para captar semántica correcta
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
embeddings = embedding_model.encode(docs, show_progress_bar=True)

# Paso 2: UMAP (Reducción de Dimensionalidad)
# n_neighbors=15: Balance entre estructura local y global
# n_components=5: Reducimos a 5 dimensiones para que HDBSCAN trabaje bien
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

# Paso 3: HDBSCAN (Clusterización)
# min_cluster_size=10: Queremos tópicos con al menos 10 reseñas
# prediction_data=True: Para poder predecir nuevos documentos luego
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# ==========================================
# 3. Entrenamiento del Modelo
# ==========================================
topic_model = BERTopic(
    embedding_model=embedding_model, # Paso 1
    umap_model=umap_model,           # Paso 2
    hdbscan_model=hdbscan_model,     # Paso 3
    language="multilingual",         # Refuerzo para stopwords en español
    calculate_probabilities=True,
    verbose=True
)

topics, probs = topic_model.fit_transform(docs, embeddings)

# ==========================================
# 4. Resultados e Interpretación de Negocio
# ==========================================

# Generar tabla de Top 5 Tópicos
freq = topic_model.get_topic_info()
print("\n--- Top 5 Tópicos Encontrados ---")
print(freq.head(6)) # head(6) porque el primero suele ser el -1 (Ruido)

# Mostrar palabras clave del Tópico 0 (el más frecuente)
print("\n--- Palabras Clave del Tópico Principal (ID 0) ---")
print(topic_model.get_topic(0))

# Visualización (Si estás en Jupyter/Colab)
topic_model.visualize_barchart(top_n_topics=5)
topic_model.visualize_topics()

Batches:   0%|          | 0/75 [00:00<?, ?it/s]

2026-01-26 16:21:07,654 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-01-26 16:21:26,748 - BERTopic - Dimensionality - Completed ✓
2026-01-26 16:21:26,749 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-01-26 16:21:26,993 - BERTopic - Cluster - Completed ✓
2026-01-26 16:21:26,996 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-01-26 16:21:27,024 - BERTopic - Representation - Completed ✓



--- Top 5 Tópicos Encontrados ---
   Topic  Count                                   Name  \
0     -1      2         -1_dañado_sucio_no_recomendado   
1      0    132         0_contento_empacado_bien_buena   
2      1    121           1_son_perfectos_compra_color   
3      2    113          2_estado_en_recomendado_llegó   
4      3    112     3_eficaz_empacado_bien_recomendado   
5      4    111  4_funciona_excelentemente_duradero_es   

                                      Representation  \
0  [dañado, sucio, no, recomendado, producto, , ,...   
1  [contento, empacado, bien, buena, rápido, de, ...   
2  [son, perfectos, compra, color, tamaño, excele...   
3  [estado, en, recomendado, llegó, rápido, excel...   
4  [eficaz, empacado, bien, recomendado, llegó, p...   
5  [funciona, excelentemente, duradero, es, el, p...   

                                 Representative_Docs  
0  [producto sucio y dañado no recomendado, produ...  
1  [rápido bien empacado y de buena calidad muy c...  


PARTE 3
Factorización Matricial (SVD)

In [13]:
import pandas as pd
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, train_test_split


# ==========================================
# 3.1 Factorización Matricial (SVD)
# ==========================================
print("=" * 40)
print("=" * 4 + " Factorización Matricial (SVD) " + "=" * 4)
print("=" * 40)

# 1. Carga y Limpieza (Eliminar outliers como el 999 detectado)
df_ratings = pd.read_csv('video_ratings.csv')
df_ratings = df_ratings[df_ratings['rating'] <= 5] # Filtramos solo valores 1-5

# 2. Configurar Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_ratings[['user_id', 'movie_id', 'rating']], reader)

# 3. Implementación de SVD y Validación Cruzada (5-fold)
algo = SVD()
results = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

rmse_promedio = results['test_rmse'].mean()
print(f"\n> RMSE Promedio tras 5-fold: {rmse_promedio:.4f}")


# ==========================================
# 3.2 El problema del Cold-Start
# ==========================================
print("=" * 40)
print("=" * 6 + " El problema del Cold-Start " + "=" * 6)
print("=" * 40)

# Dividir en entrenamiento y prueba (80/20)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
algo.fit(trainset)
predictions = algo.test(testset)

# Identificar usuarios fríos (< 3 interacciones en el set original)
user_counts = df_ratings['user_id'].value_counts()
cold_users = user_counts[user_counts < 3].index

# Filtrar predicciones de test que corresponden a usuarios fríos
cold_predictions = [p for p in predictions if p.uid in cold_users]

# Calcular RMSE para este subgrupo
rmse_cold = accuracy.rmse(cold_predictions)
print(f"RMSE para usuarios Cold-Start: {rmse_cold:.4f}")

==== Factorización Matricial (SVD) ====
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.6891  0.7252  0.6942  0.6849  0.6861  0.6959  0.0150  
MAE (testset)     0.3702  0.3805  0.3771  0.3656  0.3709  0.3729  0.0053  
Fit time          0.14    0.14    0.14    0.14    0.17    0.14    0.01    
Test time         0.02    0.02    0.02    0.02    0.03    0.02    0.00    

> RMSE Promedio tras 5-fold: 0.6959
====== El problema del Cold-Start ======
RMSE: 0.1035
RMSE para usuarios Cold-Start: 0.1035


PARTE 4.

In [27]:
import pandas as pd
import scipy.sparse as sparse
from implicit.als import AlternatingLeastSquares
import os

# Paso opcional: Evita problemas de hilos en Anaconda/Mac
os.environ['MKL_NUM_THREADS'] = '1'

# 1. Limpieza y conversión de tipos
df_music = pd.read_csv('music_logs.csv').dropna()
df_music = df_music[df_music['play_count'] > 0].copy()

# 2. Crear la matriz Ítem-Usuario directamente
# Convertimos play_count a float32 para silenciar el Warning
user_items = sparse.csr_matrix((
    df_music['play_count'].astype('float32'), 
    (df_music['song_id'].astype(int), df_music['user_id'].astype(int))
))

# 3. Entrenar el modelo
# Ya no necesitamos usar .T en fit() porque creamos user_items directamente
model = AlternatingLeastSquares(factors=64, regularization=0.1, iterations=20)
model.fit(user_items) 



  0%|          | 0/20 [00:00<?, ?it/s]